In [6]:
from selenium import webdriver
import selenium
from bs4 import BeautifulSoup
import pandas
import numpy
import time
import random
import os

In [7]:
print('-'*100)
print('NAVER 부동산 칼럼 크롤러')
print('='*100)
cnt = int(input('1.몇 건의 부동산 칼럼 데이터를 수집할까요?'))
f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+'NAVER 부동산 칼럼 크롤링 결과')
os.chdir(f_dir+s+'-'+'NAVER 부동산 칼럼 크롤링 결과')
f_name = s+'-'+'NAVER 부동산 칼럼.txt'

fc_name = s+'-'+'NAVER 부동산 칼럼 제목위주.csv'
fc_name2 = s+'-'+'NAVER 부동산 칼럼 내용위주.csv'

fx_name = s+'-'+'NAVER 부동산 칼럼 제목위주.xlsx'
fx_name2 = s+'-'+'NAVER 부동산 칼럼 내용위주.xlsx'

----------------------------------------------------------------------------------------------------
NAVER 부동산 칼럼 크롤러
1.몇 건의 부동산 칼럼 데이터를 수집할까요?300


In [8]:
path = '/Users/kibeomkim/Desktop/chromedriver_88/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('--kiosk')
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')

driver = webdriver.Chrome(path, options=options)
driver.get('https://land.naver.com/news/expertColumn.nhn?page=1')
s_time = time.time()
time.sleep(random.randrange(2,5))

In [9]:
f = open(f_name, 'a', encoding='utf-8')
no = 1
con_no = 1
page_no = 1

title_list1 = []
source_list1 = []
date_list1 = []
likes_list1 = []
search_n_list1 = []
press_name_list1 = []
contents_list1 = []

while True : 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    cont_list = soup.find('div', 'table_list_column').find('tbody').find_all('tr')
    #컨텐츠 목록 리스트 
    
    for i in cont_list : 
        #기사끼리 구분선
        print('-'*100)
        f.write('-'*100+'\n')
        
        #기사 제목 수집 
        title = i.find('td', 'title').find('a').get_text().replace('\n', "").strip()
        title_list1.append(title)
        print('1.기사 제목 : %s' %title)
        f.write('1.기사 제목 : %s' %title+'\n')
        
        #기사 출처 수집 
        source = i.find('td', 'title').find('span', 'columnist').get_text().replace('\n', "").strip()
        source_list1.append(source)
        print('2.기사 출처 : %s' %source)
        f.write('2.기사 출처 : %s' %source+'\n')
        
        #기사 게시 일자 수집
        date = i.find('td', 'title').find('span', 'date').get_text().replace('\n', "").strip()
        date_list1.append(date)
        print('3.기사 게시 일자 : %s' %date)
        f.write('3.기사 게시 일자 : %s' %date+'\n')
        
        #기사 별 좋아요 갯수 수집
        likes = i.find('td', 'like like_on').find('em', 'u_cnt _cnt').get_text().replace('\n', "").strip()
        likes = int(likes)
        likes_list1.append(likes)
        print('4.좋아요 갯수 : %s' %likes)
        f.write('4.좋아요 갯수 : %s' %likes+'\n')
        
        #기사 별 조회수 수집
        search_n = i.find('td', 'view').get_text().replace('조회', "").replace(',', "").strip()
        search_n = int(search_n)
        search_n_list1.append(search_n)
        print('5.기사 조회수 : %s' %search_n)
        f.write('5.기사 조회수 : %s' %search_n+'\n')
        
        #기사 내용 수집 
        driver.find_element_by_xpath('/html/body/div/div[4]/div[1]/div[2]/table/tbody/tr[%s]/td[1]/a' %con_no).click()
        time.sleep(3)
        
        html1 = driver.page_source
        soup1 = BeautifulSoup(html1, 'html.parser')
        
        #언론사 제목
        try : 
            press_name = soup1.find('strong', 'media_end_summary').get_text().replace('\n', "").strip()
            press_name_list1.append(press_name)
            print('6.언론사 또는 기자 이름 : %s' %press_name)
            f.write('6.언론사 또는 기자 이름 : %s' %press_name+'\n')
        except : 
            press_name_list1.append(None)
            print('6.언론사 또는 기자 이름 : 제공된 정보 없음')
            f.write('6.언론사 또는 기자 이름 : 제공된 정보 없음'+'\n')
        
        #기사 내용 
        contents = soup1.find('div', 'column_txt size4').get_text().replace('\n', "").strip()
        contents_list1.append(contents)
        print('7.기사 내용 : %s' %contents)
        f.write('7.기사 내용 : %s' %contents+'\n')
        
        #뒤로 가기 
        driver.back()
        time.sleep(5)
        
        no += 1
        con_no += 1
        
        if no > cnt : 
            break
    if no > cnt : 
        break
        
    page_no += 1
    con_no = 1 #다음 페이지 넘어가기 전에 초깃값 1로 되돌리기 
    
    try : 
        driver.find_element_by_link_text(str(page_no)).click()
        time.sleep(3)
    except : 
        try : 
            driver.find_element_by_link_text('다음페이지 ').click()
            time.sleep(random.randrange(2, 5))
        except : 
            break
            
f.close()
driver.quit()

#기사 정보 중심의 데이터프레임1
estate = pd.DataFrame({
    '기사제목' : title_list1, 
    '기사소스' : source_list1,
    '기사게시일자' : date_list1,
    '기사좋아요갯수' : likes_list1,
    '기사조회수' : search_n_list1,
    '언론사또는기자이름' : press_name_list1
})
estate_id = list(estate.index)
estate['estate_id'] = estate_id

estate.to_csv(fc_name, encoding='utf-8-sig')
estate.to_excel(fx_name)


#기사 내용 중심의 데이터프레임2
estate2 = pd.DataFrame({
    'estate_id' : estate_id,
    '기사 제목' : contents_list1
})
estate2.to_csv(fc_name2, encoding='utf-8-sig')
estate2.to_excel(fx_name2)

e_time = time.time()
total_time = e_time - s_time 


#수집 결과 알려주기 
print('-'*100)
print('요청하신 %s 건 자료 중 총 %s건 만큼 수집했습니다' %(cnt, (no-1)))
print('총 %s초가 데이터 수집에 소요되었습니다' %round(total_time, 2))
ff_dir = f_dir+s+'-'+'NAVER 부동산 칼럼 크롤링 결과'
print('파일 저장 경로는 %s/ 입니다' %ff_dir)
print('-'*100)
     
    

----------------------------------------------------------------------------------------------------
1.기사 제목 : [기자의 눈] '공공은 선'이란 허상
2.기사 출처 : 서울경제
3.기사 게시 일자 : 2021.03.31.
4.좋아요 갯수 : 12
5.기사 조회수 : 1408
6.언론사 또는 기자 이름 : 권혁준 건설부동산부 기자
7.기사 내용 : 권혁준 건설부동산부 기자[서울경제] “‘이기적 개인 대 공공의 정부’라는 가정은 환상이다. 국가와 정부를 위해 일하는 사람 역시 이기적인 동기에 따라 움직인다. 사회적 후생, 정의·복지 등을 내세우지만 실제로는 정부 기능과 규모를 키울수록 해당 부처가 이익을 얻는다. 정부는 점점 몸집을 불리고 재정 적자는 눈덩이처럼 불어난다.”‘공공선택이론’으로 1986년 노벨 경제학상을 수상한 제임스 뷰캐넌 부고 기사의 일부다. 그가 보여준 통찰이 놀랍게도 반백 년 가까이 지난 지금 문재인 정부의 정치·경제 정책에 그대로 적용되고 있다. 이들은 ‘선한 정부’를 내세우며 각종 세금을 인상하고, 그 비용으로 몸집을 거대하게 불렸다. 적자 또한 눈덩이처럼 불어났다. ‘민간은 악, 공공은 선’이라는 프레임도 뷰캐넌이 예측한 그대로다.이들이 과연 불린 몸집으로 무슨 행동을 했는가는 대한민국 국민 모두가 목격하고 있다. 한국토지주택공사(LH) 직원들은 사전 정보를 바탕으로 신도시 조성 예정 지역에 투기했다. 여당 국회의원 및 가족 이름도 다수 발견됐다. 민간 재개발·재건축이 건설사·부동산 투기꾼들의 배만 불릴 것이라며 분양가 상한제 등 각종 규제로 옥죄고, 공공 택지만을 추진한 결과다. 진짜 ‘투기꾼’은 민간이 아닌 공공에서 나왔던 것이다.‘악한 집주인’들을 막겠다며 만든 ‘임대차 3법’도 정치인·관료 개인의 이기적 욕심을 막지는 못했다. 김상조 청와대 정책실장은 법이 시행되기 직전 자기 소유의 아파트 임대료를 대폭 올렸다. 나름 ‘약자를 위한다며 나선’ 정

----------------------------------------------------------------------------------------------------
1.기사 제목 : [이규화의 지리角角]서울 재건축, 50층이냐 70층이냐
2.기사 출처 : 디지털타임스
3.기사 게시 일자 : 2021.03.26.
4.좋아요 갯수 : 0
5.기사 조회수 : 865
6.언론사 또는 기자 이름 : 제공된 정보 없음
7.기사 내용 : 35는 서울 재건축조합 주민들에게 지난 10년여간 천형(天刑)과도 같았던 숫자다. 35는 층고(층수)를 말한다. 전임 박원순 서울시장은 서울시 조례로 주거지역 아파트의 최고 층고를 35층으로 제한했다. 층고를 더 높여 아파트단지를 재건축하려던 재건축조합이나 재개발조합의 주민들은 애가 탔다. 대표적인 것이 강남구 대치동 은마아파트단지다.층고는 재건축에서 매우 중요한 요소다. 가구를 늘릴 수 있어 사업의 채산성을 높일 수 있고, 조망권을 확보하며, 건폐율을 낮춰 넓은 녹지를 가질 수 있을 뿐아니라 랜드마크 효과까지 있어 재산 가치를 높여준다. 그러니 은마아파트처럼 적절한 조건이 조성될 때까지 10년 넘게 기다리겠다는 조합이 수두룩하다. 특히 강남서초지역에 많다.◆박영선·오세훈 후보 모두 35층 규제완화 약속 그런데 박원순 마법이 풀리고 있다. 이번 서울시장 보궐선거에서 더불어민주당 박영선 후보와 범야권 국민의힘 오세훈 후보 모두 35층 층고제한을 풀겠다고 밝혔다. 서울의 신규아파트 공급을 위해서는 층고를 높여야 한다는 데 모두 공감한다. 단, 두 후보간 방식은 다르다. 박영선 후보는 층고 규제를 푸는 데는 동의하면서도 정부가 추진하는 공공이 개입하는 재건축·재개발 방식에 가깝다. 오세훈 후보는 재건축·재개발을 민간에 맡길 것이라고 한다. 오 후보는 서울시가 조례로 국가법령에서 정하는 용적률 규제보다 30%에서 100%까지 용적률을 낮게 적용하는 것은 불합리하다며 정비할 것이라고 밝혔다. 박·오 후보 모두 층고를 어느 선까지 허

----------------------------------------------------------------------------------------------------
1.기사 제목 : [기자수첩]6억 이하 재산세 깎았으니 괜찮다?…포장 급급한 정부
2.기사 출처 : 이데일리
3.기사 게시 일자 : 2021.03.19.
4.좋아요 갯수 : 4
5.기사 조회수 : 6769
6.언론사 또는 기자 이름 : 제공된 정보 없음
7.기사 내용 : [이데일리 김나리 기자] 올해 전국 공동주택 공시가격 평균 상승률이 19.8%로 14년 만에 가장 큰 폭으로 뛰면서 세 부담을 우려하는 목소리가 커지고 있다.공시가 급등에 따른 논란이 거세지자 국토부는 “공시가 6억원 이하 1주택자에 대해서는 재산세율을 0.05%포인트 낮췄기 때문에 오히려 재산세 부담이 줄어든다”고 해명했다. 전국 공동주택 92.1%를 차지하는 공시가 6억 이하는 공시가가 오르더라도 재산세는 지난해보다 더 줄어든다는 설명이다.[이데일리 노진환 기자]앞서 정부는 지난해 서민층 조세 부담을 줄이기 위해 1가구 1주택자의 공시가 6억원 이하 주택의 재산세율을 2023년까지 3년간 한시적으로 완화하겠다는 방침을 밝혔다. 2030년까지 공시가 현실화율(시세반영율)을 90%까지 끌어올리기로 하면서 이에 따른 세 부담 우려가 커지자, 공시가 6억원 이하 주택(1가구 1주택자) 재산세율은 올해부터 3년간 0.05%포인트씩 인하해 세 부담을 줄여주겠다고 한 것이다. 국토부는 예를 들어 서울 관악구 A아파트 전용면적 84㎡는 공시가격이 4억9700만원에서 5억9200만원으로 19.1% 올랐으나 재산세는 105만1000원에서 94만2000원으로 10.4% 내려갈 것으로 추산했다. 그러나 정부가 강조한 ‘공시가 6억원 이하 주택’ 비중은 점차 감소하는 추세다. 실제 국토교통부가 집계한 시·도 공시가격별 공동주택 분포 통계를 보면 공시가 6억원 이하 전국 공동주택 비중은 2019년 96.1%에서 지난해 95.1%, 올해 

----------------------------------------------------------------------------------------------------
1.기사 제목 : [김은희의 현장에서] 2030이 LH 사태에 분노하는 이유
2.기사 출처 : 헤럴드경제
3.기사 게시 일자 : 2021.03.11.
4.좋아요 갯수 : 3
5.기사 조회수 : 4007
6.언론사 또는 기자 이름 : 제공된 정보 없음
7.기사 내용 : “내 집 하나 마련하겠다고 전전긍긍했는데 허탈합니다.”“혹시나 믿었는데 역시나 그들만의 리그였군요. ‘벼락거지’에 바보까지 됐어요.”한국토지주택공사(LH) 직원들의 광명·시흥신도시 땅 투기 의혹에 2030 무주택자들이 박탈감을 호소하고 있다. 흔히 연애·결혼·출산은 물론 내 집 마련과 서울 거주까지 포기한 ‘N포 세대’로 불리는 이들이다. 웬만한 시련에도 끄떡없을 정도로 맷집이 단단해졌다지만 이번 논란에는 타격이 상당한 분위기다. 2·4공급대책 발표 당시 정부가 내놓은 확언에 대한 일종의 배신감으로 읽힌다.2030 무주택자들은 문재인 정부가 스물네 번의 부동산 정책을 내놓을 때마다 기대와 실망을 반복해왔다. 대책을 낼 때마다 시장이 가라앉기는커녕 요동쳤다. 집값은 치솟았고 내 집 마련의 꿈은 멀어져 갔다. 정부가 25번째 대책을 예고했을 때 의심의 눈초리를 보냈던 이유를 알 만하다.2·4대책은 ‘공공과 물량, 속도’라는 세 가지 키워드로 요약된다. 정부가 직접, 빠르게, 많이 주택을 공급하겠다는 것이다. 모든 공급 정책이 그렇듯 당장 시장에 반영되긴 어려운 내용이다. 그런데도 시장에선 어느 정도 반응이 왔다. 폭발했던 수요가 일부 관망세로 돌아선 것이다. 거래량이 줄고 매물이 늘고 가격 상승폭이 일부 줄었다는 통계가 이를 방증한다. 물론 2·4대책의 효과라고만 단언할 수 없다. 다만 긍정적인 변화의 가능성을 봤다는 데에는 이견이 없다.‘정부가 책임지고 할 테니 이번만은 믿어달라’는 절박한 호소가 통했는지도 모른다. 정부

----------------------------------------------------------------------------------------------------
1.기사 제목 : [톺아보기] 차질없는 공급계획 추진을 위한 과제
2.기사 출처 : 아시아경제
3.기사 게시 일자 : 2021.03.03.
4.좋아요 갯수 : 0
5.기사 조회수 : 1007
6.언론사 또는 기자 이름 : 김덕례 주택산업연구원 선임연구위원
7.기사 내용 : 김덕례 주택산업연구원 선임연구위원차질없는 공급계획 추진을 위한 과제 - 김덕례 주택산업연구원 선임연구위원모든 국민이 집 걱정 없이 살 수 있는 대한민국을 만들기 위한 공급정책이 속도를 내고 있다. 서울 32만호를 포함해 대도시권에 83만 6000호의 주택이 공급된다. 이를 위해 2·4대책이 담고 있는 공급수단은 정비사업, 도심공공주택복합사업(역세권·준공업·저층주거), 소규모 도시재생, 비주택리모델링, 신축매입 그리고 공공택지까지 매우 다양하다. 수도권 신도시 개발도 박차를 가하고 있다. 남양주왕숙, 하남교산, 고양창릉, 부천대장, 인천계양과 더불어 광명시흥지구가 6번째 신도시로 추가된다.2·4대책에서 발표한 주택 공급물량과 그동안 주거복지로드맵, 3기 신도시 등을 통해 발표한 물량을 모두 합치면 200만호 수준에 이른다. 분당, 일산 등 1기 신도시를 공급했던 1990년대 200만호 주택공급계획과 유사한 규모다.국민은행 가격지수를 기준으로 1989년과 1990년에 전국주택가격은 15%~20% 상승했다. 30%가 넘게 급등하는 지역도 있었다. 그러나 1기 신도시가 본격적으로 공급되면서 1991년부터 1995년까지 5년간 전국적으로 주택가격이 하락했다. 주택공급효과다. 정부의 공급계획이 차질없이 추진되면 기대할 수 있는 효과이기도 하다.그러기 위해 해결해야 할 몇 가지 과제가 있다. 우선 부지확보기준이 갖는 시차문제다. 2·4대책에서 발표한 주택물량이 시장에 분양공급되려면 상당한 시간이 걸린다. 그때까지 사람들의 불안

...

----------------------------------------------------------------------------------------------------
1.기사 제목 : [시론] 5년 하락, 5년 상승 … 수도권 주택시장 변곡점 왔나
2.기사 출처 : 중앙일보
3.기사 게시 일자 : 2018.03.21.
4.좋아요 갯수 : 8
5.기사 조회수 : 25031
6.언론사 또는 기자 이름 : 수도권 주택시장 갑론을박아파트시장에도 경기 사이클정부와 시장, 강남에 조급증장기적 관점서 시장을 봐야
7.기사 내용 : 수도권 주택시장 갑론을박아파트시장에도 경기 사이클정부와 시장, 강남에 조급증장기적 관점서 시장을 봐야  허윤경 한국건설산업연구원·주택도시연구실 연구위원     수도권 주택시장이 급변하고 있다. 작년 11월부터 가파른 상승세를 이어오던 수도권 아파트 가격이 3월 들어 상승 폭이 둔화하고 거래량은 감소하고 있다. 전셋값은 하락세로 돌아섰다. 지금까지 가격 급등을 주도한 강남·서초·강동구의 전셋값도 하락세다. 정책 방향의 변화도 강하다. 이런 현상을 두고 갑론을박이 많다. 정부의 8·2 부동산 대책 이후 계속된 시장과 규제의 힘겨루기 와중에 정책 효과가 발휘됐다고 보는 시각이 있다. 반면 일시적 현상으로 해석하는 시각도 있다. 미래를 예측하는 것은 어렵다. 누구나 알고 있지만 쉽게 지나치는 주택시장의 근본적인 특징을 살펴보면 힌트를 얻을 수 있다.      먼저, 주택시장은 경기 사이클이 존재한다. 긴 호흡으로 상승과 하락을 반복한다. 강남도 마찬가지다. 수도권 주택시장은 2013년 초부터 상승하기 시작해 지금까지 5년간 상승했다. 하지만, 이는 2008년 금융위기 이후 5년 동안 하락한 뒤 나타난 현상이다. 지난 10년간의 주택경기 사이클이 되풀이된다면 현 시점이 변곡점일 가능성이 높아 예의주시해야 한다. 상승이 있으면 하락이 있게 마련이다.      주택시장엔 가격의 단기적 등락이 존재하지만, 장기적으로 펀더멘탈에 수렴한다. 거시경제

----------------------------------------------------------------------------------------------------
1.기사 제목 : [비즈 칼럼] 재건축 남용 막기 위한 안전진단 강화
2.기사 출처 : 중앙일보
3.기사 게시 일자 : 2018.03.12.
4.좋아요 갯수 : 6
5.기사 조회수 : 8585
6.언론사 또는 기자 이름 : 제공된 정보 없음
7.기사 내용 : 조명래 단국대 도시지역계획학과 교수     아파트 안전진단에서 ‘안전하지 않은 등급’을 받으면 이를 축하하는 펼침 막이 내걸린다. 우리는 자기 집이 안전하지 않은 것을 축하하는 이상한 나라에 살고 있다. 상식적으로 이해하기 힘들지만 모든 게 돈 때문이다. 안전진단을 통과하면 낡은 집을 허물고 돈이 되는 새집을 짓는 게 가능해진다. 재건축(을 기다리는) 아파트의 몸값이 금값인 까닭은 여기에 있다. 이러니 집을 보수해 오래 사용하는 것에는 관심이 없다. 거꾸로 안전하지 않은 것을 증빙하기 위해 건축물을 의도적으로 방치하고 손쉬운 재건축을 위해 안전진단 기준 완화가 늘 요구받는다.      한국의 아파트 수명이 짧고 멀쩡한 집을 허물어 새로 짓는 재건축이 남용되고 투기화되는 것은 우리의 독특한 재건축 제도와 관련된다. 재건축 안전진단제도는 낡아서 구조적으로 문제가 있는 아파트에 대해 재건축을 허용하기 위해 2003년 도입됐다. 평가항목은 구조 안전성, 주거환경, 비용 편익, 설비 노후도 네 가지다. 가장 중요하면서 통과하기 어려운 항목이 구조 안전성이다. 재건축 용이성을 좌우하는 이 항목의 비중 변화는 역대 정부의 부동산 규제와 직결돼 있다. 실제 2003년 45%에서 2006년 50%로 올랐다가 2009년 40%, 2015년 20%로 축소됐다.      2015년 기준 완화는 2014년 9·1 대책의 재건축 규제 완화 일환이었다. 구조 안전성의 비중을 40%→20%로 낮춘 반면, 주거환경비중을 15%→40%로 높이게 되면서 안전진단

----------------------------------------------------------------------------------------------------
1.기사 제목 : [목멱칼럼]일본에서 배워야 할 스마트시티 수출 노하우
2.기사 출처 : 이데일리
3.기사 게시 일자 : 2018.02.28.
4.좋아요 갯수 : 1
5.기사 조회수 : 2897
6.언론사 또는 기자 이름 : 제공된 정보 없음
7.기사 내용 : 심교언 건국대 부동산학과 교수.[심교언 건국대 부동산학과 교수] 지난 1월 정부는 스마트시티 국가시범도시로 세종시 일부와 부산시 에코델타시티를 지정했다. 앞으로 다가올 4차 산업 혁명에 적극적으로 대비하고 이를 기회로 혁신적인 산업 생태계를 조성하고자 함이다. 정부 계획대로 된다면 새로운 산업의 주도권을 우리가 잡게 되고 이를 통한 일자리 창출도 기대된다. 건설과 정보통신기술(ICT) 등을 융·복합해 다양한 도시 서비스를 제공하는 스마트시티는 과거와 같이 자본이 많이 투입되는 물리적 인프라 구축이 아니라, 정보를 제공해 투자 대비 효율성을 극대화하는 경우가 많다. 교통시스템의 경우 영국 고속도로에서 도입한 결과 통행시간이 25%, 교통사고 50%, 대기오염 10% 감축이라는 놀라운 결과를 얻었고, 베이징 등지에서는 40~50% 정도의 누수 예방 효과를 보았다. 바르셀로나도 연간 30% 정도의 에너지 절감효과가 있었다고 한다. 이러한 장점을 바탕으로 스마트시티 시장 규모는 연평균 약 13.9%씩 성장해 2020년에는 2조100억달러에 달할 것으로 전망되고 있다. 우리나라도 세계 최고 수준의 IT 인프라를 바탕으로 비교적 일찍 ‘U시티’ 사업을 비롯한 다양한 정책을 펴서 초기에는 선도적 지위를 구축했다. 하지만 지속가능성이라던가 거버넌스 등의 분야에서는 경쟁력이 하락하는 것으로 평가받고 있어 안타깝다. 최근 도시 개발과 관련한 해외 원조사업에서 수원국들이 스마트시티에 대한 관심을 많이 표명하고 있어서 한국 기업 진출의 새로운 기회

----------------------------------------------------------------------------------------------------
1.기사 제목 : [전문가칼럼]3억 이하로 내 집 마련하기
2.기사 출처 : 동아일보
3.기사 게시 일자 : 2018.02.22.
4.좋아요 갯수 : 8
5.기사 조회수 : 11008
6.언론사 또는 기자 이름 : 제공된 정보 없음
7.기사 내용 : 곽창석 도시와공간 대표서울 아파트값 상승세가 심상치 않다. 지난해 12월 이후 계약한 건의 상당수가 계약금보다 집값이 더 올라 집주인이 해약하는 사례가 꽤 많다. 서울 강남에서 시작한 상승세는 마포 용산 성동을 거쳐 강북지역과 서울 인접 외곽지역으로 확산되고 있다.올봄에 만기가 도래하는 무주택자는 또다시 집값 상승의 불안감에 휩싸이고 있다. 전세를 연장하자니 집값이 오를 것 같고, 집을 구입하자니 마음에 드는 집의 가격은 저 멀리 도망가 있다. 한때는 청약해서 당첨되는 꿈을 꿨지만 청약가점제가 전면적으로 시행되고 특별공급이 확대되면서 당첨의 꿈도 접어야 할 판이다.전세 대출을 더 받아서 한번 더 전세를 살지, 아니면 돈에 맞춰서 다세대 주택이라도 구입할지 고민스럽다. 다세대 주택은 집값이 안 오르니 구입하지 말라고 말리는 주변 사람들이 많아 선뜻 손이 가지 않는다. 가진 돈에 맞는 아파트를 찾아보지만 성에 차지 않는다. 낡았거나 교통이 불편하거나 편의시설이 없거나 학군이 나쁘거나 등등 살만한 아파트가 없다. 내 눈에 들어오는 아파트는 내 능력에 비해 1~2억은 더 있어야 구입이 가능할 뿐이다. 할 수 없이 전세를 연장하거나 새로운 전세를 찾는 방법 밖에는 선택의 여지가 없다고 결정하게 된다. 앞으로 전셋값이 오르지 말아달라고 기도하면서.어떤 면에선 전세를 고집하는 사람들의 공통점은 완벽주의자이거나 욕심이 넘치는 경우가 많다. 내가 구입할 집은 지금보다 더 훌륭한 집이어야 한다는 꿈을 꾸면서 집 구입을 미룬다. 더 싸게 살 기회가 앞으로 

----------------------------------------------------------------------------------------------------
1.기사 제목 : [기자의 눈]장관의 발언이 신중해야 하는 이유
2.기사 출처 : 서울경제
3.기사 게시 일자 : 2018.02.06.
4.좋아요 갯수 : 3
5.기사 조회수 : 3116
6.언론사 또는 기자 이름 : 제공된 정보 없음
7.기사 내용 : 고병기 기자  <건설부동산부>[서울경제] 김현미 국토교통부 장관의 발언 한마디 한마디가 연일 뜨거운 이슈가 되고 있다. 김 장관이 국민의 최대 관심사 중 하나인 주택시장에 큰 파장을 일으킬 만한 말들을 내뱉었다 주워담는 일이 반복되고 있기 때문이다. 김 장관은 6일 국회 국토교통위원회 전체회의에 출석해 “재건축 연한에 대해 30년이나 40년을 언급한 적이 없는데 마치 그런 말을 한 것처럼 나와서 의아하고 당혹스럽다”고 했다. 그간 시장에서는 김 장관이 재건축 연한을 40년으로 연장하는 것을 시사했다고 이해하고 있었는데 이를 처음으로 공식 부인한 것이다. 사람들이 김 장관의 발언을 재건축 연한 연장으로 해석한 것은 지난달 18일 서울 가좌 행복주택에서 열린 주거복지협의체에서 나온 김 장관의 발언 때문이다. 당시 정부의 고강도 규제책에도 불구하고 서울 주택시장의 과열 양상이 계속되자 기자들이 재건축 연한을 연장할 것이냐는 질문을 했고 이에 대해 김 장관은 “재건축은 주거환경을 개선하는 순기능이 있는 것은 사실이지만 구조 안전성의 문제가 없음에도 사업 이익을 얻기 위해 사회적 자원을 낭비한다는 문제 제기가 있는 것도 사실”이라며 “건축물 구조적 안정성이나 내구연한 등의 문제를 종합적으로 검토해볼 필요가 있지 않나 생각한다”고 답했다. 이날 김 장관의 해명대로 논란이 되고 있는 재건축 연한 30년이나 40년에 대해서는 전혀 언급하지 않았지만 대부분의 사람들은 맥락상 정부가 재건축 연한 연장을 검토하는 것으로 받아들였다. 이후 언론은 정부의 